# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = 'T9.AQT87.873.653'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['PAD_START', "PASS", "1H", "X"]

# what's your bid?

In [4]:
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, -1, -1, 0.1, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': '4H', 'insta_score': 0.4037, 'expected_score': 90, 'adjustment': 20},
 {'call': '2H',
  'insta_score': 0.5911,
  'expected_score': -104,
  'adjustment': 30}]

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(["PAD_START", "PASS", "1H", "X", "4H", "PASS", "PASS", "X"])
bid.bid

Bid not valid 3H insta_score: 0.9810969829559326


'PASS'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.0153}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['T9.AQT8x.8xx.xxx J8xxx.xx.JTx.K98 x.KJ9xx.AQxx.JTx AKQxx.x.K9x.AQxx',
 'T9.AQT8x.8xx.xxx Qxx.xx.QT9.T98xx J8x.KJ9xx.AJxx.Q AKxxx.x.Kxx.AKJx',
 'T9.AQT8x.8xx.xxx 8xx.x.JT9xx.T8xx KQx.KJ9xxx.Qx.Qx AJxxx.x.AKx.AKJ9',
 'T9.AQT8x.8xx.xxx J8xx.9xx.QTxx.Tx KQx.KJxxx.9x.KQx Axxx..AKJx.AJ98x',
 'T9.AQT8x.8xx.xxx xxxx.x.9xxx.J98x Kx.J9xxx.AQJx.Qx AQJ8x.Kx.KT.AKTx',
 'T9.AQT8x.8xx.xxx xxxxx.Jx.9x.KQTx QJ.K9xxx.AQxx.xx AK8x.x.KJTx.AJ98',
 'T9.AQT8x.8xx.xxx xxxx.xx.Qxx.T9xx x.KJ9xxx.ATx.AQJ AKQJ8x..KJ9x.K8x',
 'T9.AQT8x.8xx.xxx Jxxx.xx.Q9x.JT9x Kxx.KJ9xx.AJx.Qx AQ8x.x.KTxx.AK8x',
 'T9.AQT8x.8xx.xxx xxxx.Kx.JTxx.98x Jxx.J9xxx.AKQ.Qx AKQ8.x.9xx.AKJTx',
 'T9.AQT8x.8xx.xxx 8xxx.xx.QJxx.9xx AKQx.KJ9xx.9x.JT Jxx.x.AKTx.AKQ8x',
 'T9.AQT8x.8xx.xxx 8xxxx.xx.9x.QJT9 AQJ.KJ9xx.JTx.xx Kxx.x.AKQxx.AK8x',
 'T9.AQT8x.8xx.xxx Q8xxx.xx.Txx.T9x Jxx.KJ9xx.Qxx.KQ AKx.x.AKJ9.AJ8xx',
 'T9.AQT8x.8xx.xxx 8xxxx.9x.QJx.JTx Jxx.KJxxx.AK.98x AKQ.x.T9xxx.AKQx',
 'T9.AQT8x.8xx.xxx 8xx.9x.Q9xx.T9xx Jx.KJxxxx.ATx.Ax AKQxxx..KJx

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS','3D'])
print(sample_hands.shape[0])
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

116
T9.AQT8x.8xx.xxx KQxx.xx.xxx.KJ8x A8xx.J9x.T9x.9xx Jxx.Kxx.AKQJ.AQT
T9.AQT8x.8xx.xxx 8xx.J9xx.QTx.AQx KJxxx.xx.J9x.8xx AQx.Kx.AKxx.KJT9
T9.AQT8x.8xx.xxx KQJx.xxx.J9.J98x 8xxx.J9x.Txx.Axx Axx.Kx.AKQxx.KQT
T9.AQT8x.8xx.xxx 8xxx.Kxxx.x.KJ8x AQxx.xx.T9xxx.Tx KJx.J9.AKQJ.AQ9x
T9.AQT8x.8xx.xxx QJxx.J9xx.Jxx.8x 8xxx.Kx.QT9.K9xx AKx.xx.AKxx.AQJT
T9.AQT8x.8xx.xxx Qxxx.J9xx.KT9x.9 J8xx.x.QJxx.AT8x AKx.Kxx.Ax.KQJxx
T9.AQT8x.8xx.xxx Q8xx.Jxxx.J9.T9x KJxx.x.QTxx.KJ8x Axx.K9x.AKxx.AQx
T9.AQT8x.8xx.xxx 8xxx.K9x.Q9xx.T9 QJxxx.xx.KT.QJ8x AK.Jxx.AJxx.AKxx
T9.AQT8x.8xx.xxx Qxxxx.J9xx.xx.Jx AJx.xx.A9xx.T9xx K8x.Kx.KQJT.AKQ8
T9.AQT8x.8xx.xxx xxxx.Kxx.9xxxx.J KJ8x.J9.QT.K98xx AQx.xxx.AKJ.AQTx
T9.AQT8x.8xx.xxx KJxxx.9xxx.KJT.T Q8x.x.9xx.QJ9xxx Axx.KJx.AQxx.AK8
T9.AQT8x.8xx.xxx Qxxxx.K9xx.Q9x.8 K8xx.xx.Kx.JT9xx AJ.Jx.AJTxx.AKQx
T9.AQT8x.8xx.xxx QJxx.9xxx.Jxxx.9 A8xxx.J.Qx.QJTxx Kx.Kxx.AKT9.AK8x
T9.AQT8x.8xx.xxx Jxxxx.Kxxx.Tx.QJ KQxx.J9.Jxxx.9xx A8.xx.AKQ9.AKT8x
T9.AQT8x.8xx.xxx Axxx.Jxxx.QJx.8x QJ8xx.9x.x

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['AQT74.43.72.A765',
 'K3.Q752.K865.KQ8',
 'J92.T986.AQJ94.4',
 '865.AKJ.T3.JT932']

In [10]:
bidder_bots = [BotBid([False, False], hand, models, -1, -1, 0.1, sampler, False) for hand in hands]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS', '1D', 'PASS', '1N', '2S', 'PASS', 'PASS', 'PASS']